In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

from environments import ARESEAOptimization

In [13]:
def pack_dataframe(fn):
    def wrapper(env, desired):
        observations, beam_images = fn(env, desired)
        observations = np.array(observations)

        df = pd.DataFrame(np.arange(len(observations)), columns=["step"])
        df["q1"] = observations[:,0]
        df["q2"] = observations[:,1]
        df["cv"] = observations[:,2]
        df["q3"] = observations[:,3]
        df["ch"] = observations[:,4]
        df["mup_x"] = observations[:,5]
        df["mup_y"] = observations[:,6]
        df["sigmap_x"] = observations[:,7]
        df["sigmap_y"] = observations[:,8]
        df["mu_x"] = observations[:,9]
        df["mu_y"] = observations[:,10]
        df["sigma_x"] = observations[:,11]
        df["sigma_y"] = observations[:,12]
        df["beam_image"] = beam_images

        df["measure_beam"] = env.backend._beam_parameter_method

        return df
    
    return wrapper

In [14]:
@pack_dataframe
def run(env, desired):
    env.next_initial = [10, -10, 10, 0, 0]
    env.next_desired = desired

    observations = []
    beam_images = []

    observation = env.reset()

    observations.append(observation)
    beam_images.append(env.backend.last_beam_image)

    def optfn(actuators):
        observation, objective, _, _ = env.step(actuators)
        observations.append(observation)
        beam_images.append(env.backend.last_beam_image)
        return objective

    minimize(optfn, observation[:5], method="Nelder-Mead", options={"disp": True})

    return observations, beam_images

In [15]:
env = ARESEAOptimization(backend="machine", backendargs={"measure_beam": "us"})

In [16]:
df = run(env, np.zeros(4))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 95
         Function evaluations: 221
